# StackGAN

In [1]:
import os
import pickle
import math

import matplotlib.pyplot as plt

from PIL import Image
import numpy as np 
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as backend 
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Input, BatchNormalization, Reshape, UpSampling2D,LeakyReLU,concatenate, Lambda, Concatenate, ReLU,ZeroPadding2D, add
from tensorflow.keras.optimizers import Adam

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
backend.set_session(sess)

In [2]:
def load_embedings(embeding_filepath):
    with open(embeding_filepath,"rb") as f:
        embedings = pickle.load(f,encoding="latin")
        
    a_embeding = np.array(embedings)
    
    embeddings = []
    for i in range(a_embeding.shape[0]):
        a1_embeding = a_embeding[i,:,:]
        ix = np.random.randint(0, a1_embeding.shape[0]-1)
        embedding = a1_embeding[ix,:]
        
        embeddings.append(embedding)
    
    
    return np.array(embeddings)

def load_images(images_path):
    with open(images_path,"rb") as f:
        images = pickle.load(f)
    
    return images

def ca_model(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = backend.exp(log_sigma)
    epsilon = backend.random_normal(shape=backend.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c

def Compressed_embed():
    input_layer = Input((1024,))
    
    x = Dense(128)(input_layer)
    x = ReLU()(x)
    
    model = Model(inputs=[input_layer], outputs=[x])
    return model

def Combined_model(g_model, d_model):
    input_layer = Input((1024,))
    input_layer2 = Input((100,))
    input_layer3 = Input((4,4,128))
    
    x, mean_logsigma = g_model([input_layer, input_layer2])
    d_model.trainable = False
    v = d_model([x, input_layer3])
    
    model = Model(inputs=[input_layer, input_layer2, input_layer3], outputs=[v, mean_logsigma])
    
    return model

In [3]:
def Generator():
    input_layer = Input((1024,))
    
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)
    
    c = Lambda(ca_model)(mean_logsigma)
    
    input_layer2 = Input((100,))
    
    gen_input = Concatenate(axis=1)([c, input_layer2])

    x = Dense(4*4*128*8, use_bias=False)(gen_input)
    x = Activation('relu')(x)
    x = Reshape((4,4,128*8), input_shape=(128*8*4*4,))(x)
    x = UpSampling2D((2, 2), data_format="channels_last")(x)
    x = Conv2D(512,kernel_size=3,padding='same', data_format="channels_last", kernel_initializer='he_normal')(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(128,kernel_size=3,padding='same', data_format="channels_last", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(64,kernel_size=3,padding='same', data_format="channels_last", kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2,2))(x)
    x = Conv2D(3,kernel_size=3,padding='same', data_format="channels_last", kernel_initializer='he_normal')(x)
    x = Activation('tanh')(x)
    
    model = Model(inputs=[input_layer,input_layer2], outputs=[x, mean_logsigma])
    
    return model

In [4]:
def Discriminator():
    input_layer1 = Input((64,64,3))
    x = Conv2D(64,kernel_size=(4,4),
                            strides=(2,2),
                            padding='same',
                            data_format="channels_last",
                            input_shape=(64,64,3),
                            use_bias=False
                            )(input_layer1)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(128, kernel_size=(4,4), padding="same",strides=(2,2), data_format="channels_last",kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(256, kernel_size=(4,4), padding="same",strides=(2,2), data_format="channels_last",kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(512, kernel_size=(4,4), padding="same",strides=(2,2), data_format="channels_last",kernel_initializer='he_normal')(x) 
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    input_layer2 = Input((4,4,128))
    marge_input = concatenate([x, input_layer2],)
    
    x2 = Conv2D(128, kernel_size=(4,4), padding="same",strides=(2,2), data_format="channels_last",kernel_initializer='he_normal')(marge_input)
    x2 = LeakyReLU(0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)
    
    model = Model(inputs=[input_layer1, input_layer2], outputs=[x2])
    
    return model

# Stage2

In [5]:
def joint_block(inputs):
    c = inputs[0]
    x = inputs[1]

    c = tf.expand_dims(c, axis=1)
    c = tf.expand_dims(c, axis=1)
    c = tf.tile(c, [1, 16, 16, 1])
    
    con = tf.concat([c,x], 3)#([c,x],axis=3)
    return con

def residual_block(input):
    """
    Residual block in the generator network
    """
    x = Conv2D(128 * 4, kernel_size=(3, 3), padding='same', strides=1)(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(128 * 4, kernel_size=(3, 3), strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    x = add([x, input])
    x = ReLU()(x)

    return x

def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + backend.exp(2. * logsigma) + backend.square(mean))
    loss = backend.mean(loss)
    return loss

In [6]:
def Generator2():
    input_layer = Input(shape=(1024,))
    input_images = Input(shape=(64,64,3))
    
    ca = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(ca)
    c = Lambda(ca_model)(mean_logsigma)
    
    x = ZeroPadding2D(padding=(1,1))(input_images)
    x = Conv2D(128, kernel_size=(3,3), strides=1, use_bias=False)(x)
    x = ReLU()(x)
    
    x = ZeroPadding2D(padding=(1, 1))(x)
    x = Conv2D(256, kernel_size=(4, 4), strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(x)
    x = Conv2D(512, kernel_size=(4, 4), strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    print(x)
    c_comb = Lambda(joint_block)([c,x])
    
    x = ZeroPadding2D(padding=(1, 1))(c_comb)
    x = Conv2D(512, kernel_size=(3, 3), strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = residual_block(x)
    x = residual_block(x)
    x = residual_block(x)
    x = residual_block(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation('tanh')(x)
    
    model = Model(inputs=[input_layer, input_images], outputs=[x, mean_logsigma])
    return model

In [7]:
def Discriminator2():
    input_layer = Input((256,256,3))
    
    x = Conv2D(64,(4,4), padding='same', strides=2, input_shape=(256,256,3),use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(64, (4, 4), padding='same', strides=2, input_shape=(256, 256, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(1024, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(2048, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(1024, (1, 1), padding='same', strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (1, 1), padding='same', strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    
    x2 = Conv2D(128, (1, 1), padding='same', strides=1, use_bias=False)(x)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)

    x2 = Conv2D(128, (3, 3), padding='same', strides=1, use_bias=False)(x2)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)

    x2 = Conv2D(512, (3, 3), padding='same', strides=1, use_bias=False)(x2)
    x2 = BatchNormalization()(x2) 
    
    added_x = add([x,x2])
    added_x = LeakyReLU(alpha=0.2)(added_x)

    input_layer2 = Input(shape=(4, 4, 128))
    
    merged_input = concatenate([added_x, input_layer2])

    x3 = Conv2D(64 * 8, kernel_size=1, padding="same", strides=1)(merged_input)
    x3 = BatchNormalization()(x3)
    x3 = LeakyReLU(alpha=0.2)(x3)
    x3 = Flatten()(x3)
    x3 = Dense(1)(x3)
    x3 = Activation('sigmoid')(x3)

    stage2_dis = Model(inputs=[input_layer, input_layer2], outputs=[x3])
    return stage2_dis

def adv_model(gen_model2, dis_model, gen_model1):
    embeddings_input_layer = Input(shape=(1024, ))
    noise_input_layer = Input(shape=(100, ))
    compressed_embedding_input_layer = Input(shape=(4, 4, 128))

    gen_model1.trainable = False
    dis_model.trainable = False

    lr_images, mean_logsigma1 = gen_model1([embeddings_input_layer, noise_input_layer])
    hr_images, mean_logsigma2 = gen_model2([embeddings_input_layer, lr_images])
    valid = dis_model([hr_images, compressed_embedding_input_layer])
    
    model = Model(inputs=[embeddings_input_layer, noise_input_layer, compressed_embedding_input_layer], outputs=[valid, mean_logsigma2])
    return model

In [8]:
BATCH_SIZE = 32
NUM_EPOCH = 100
GENERATED_IMAGE_PATH = "generated_images/"
G_WEIGHTS_SAVEPATH = "generator.h5"
D_WEIGHTS_SAVEPATH = "discriminator.h5"

r_loss = []
f_loss = []
d_loss = []

def train():
    x_train = load_images("./generate_images/64images.pickle")
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
    embeddings = load_embedings("./generate_images/char-CNN-RNN-embeddings.pickle")
    embeddings_test = load_embedings("./flowers/test/char-CNN-RNN-embeddings.pickle")
    
    discriminator = Discriminator()
    d_opt = Adam(lr=0.00015, beta_1=0.5, beta_2=0.999)
    discriminator.compile(loss="binary_crossentropy", optimizer=d_opt)
    
    discriminator.trainable = False
    generator = Generator()
    g_opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)
    combined_model = Combined_model(generator, discriminator)
    combined_model.compile(loss='binary_crossentropy', optimizer=g_opt)
    #dcgan = Sequential([generator, discriminator])
    #g_opt = Adam(lr=2e-4, beta_1=0.5)
    #dcgan.compile(loss='binary_crossentropy', optimizer=g_opt)
    
    compressed_embed = Compressed_embed()
    compressed_embed.compile(loss='binary_crossentropy', optimizer="adam")
    
    if os.path.exists(G_WEIGHTS_SAVEPATH):
        generator.load_weights(G_WEIGHTS_SAVEPATH)
        
    if os.path.exists(D_WEIGHTS_SAVEPATH):
        discriminator.load_weights(D_WEIGHTS_SAVEPATH)
    
    num_batches = int(x_train.shape[0] / BATCH_SIZE)
    print('Number of batches:', num_batches)
    for epoch in range(NUM_EPOCH):
        for index in range(num_batches):
            #noise = np.array([np.random.uniform(0,1,100) for _ in range(BATCH_SIZE)])
            noise = np.random.randn(BATCH_SIZE, 100)
            image_batch = x_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            embedding_batch = embeddings[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images, _ = generator.predict([embedding_batch, noise], verbose=0)
            
            if index % 100 == 0:
                z_noise2 = np.random.normal(0, 1, size=(BATCH_SIZE, 100))
                embedding_batch = embeddings_test[0:32]
                fake_images, _ = generator.predict([embedding_batch, z_noise2])
                fake_images = fake_images*125.7 + 127.5
                # Save image
                Image.fromarray(np.uint8(fake_images[0])).save("./generate_images/images4/%depoch_%d.jpg"%(epoch, index))
            
            compressed_embeddings = compressed_embed.predict_on_batch(embedding_batch)
            compressed_embeddings = np.reshape(compressed_embeddings, (-1, 1, 1, 128))
            compressed_embeddings = np.tile(compressed_embeddings, (1, 4, 4, 1))
            
#             X = np.concatenate((image_batch, generated_images))
#             comp_emb = np.concatenate((compressed_embeddings, compressed_embeddings))
            real_label = np.ones((BATCH_SIZE, 1)) * 0.9
            fake_label = np.ones((BATCH_SIZE, 1)) * 0.1
            real_d_loss = discriminator.train_on_batch([image_batch, compressed_embeddings], np.reshape(real_label, (BATCH_SIZE, 1)))
            fake_d_loss = discriminator.train_on_batch([generated_images, compressed_embeddings],np.reshape(fake_label, (BATCH_SIZE, 1)))
            
            r_loss.append(real_d_loss)
            f_loss.append(fake_d_loss)
            d_loss.append(real_d_loss + fake_d_loss)
            
            #noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            noise = np.random.randn(BATCH_SIZE, 100)
            g_loss = combined_model.train_on_batch([embedding_batch, noise, compressed_embeddings], [np.ones((BATCH_SIZE,1)) * 0.9, np.ones((BATCH_SIZE,256)) * 0.9])
            #g_loss = dcgan.train_on_batch([embedding_batch, noise] ,[1]*BATCH_SIZE)
        #print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" %(epoch, index, g_loss, d_loss))
            print("epoch: %d, batch: %d" %(epoch, index))

        generator.save_weights("1023"+G_WEIGHTS_SAVEPATH)
        discriminator.save_weights("1023"+D_WEIGHTS_SAVEPATH)

#         generator.save_weights(G_WEIGHTS_SAVEPATH)
#         discriminator.save_weights(D_WEIGHTS_SAVEPATH)
#         plt.plot(range(NUM_EPOCH), r_loss, label="real_loss")
#         plt.plot(range(NUM_EPOCH), f_loss, label="fake_loss")
#         plt.plot(range(NUM_EPOCH), d_loss, label="comb_loss")
#         plt.legend()
#         plt.show()
        

In [9]:
BATCH_SIZE =2
NUM_EPOCH = 2
GENERATED_IMAGE_PATH = "generated_images/"
G_WEIGHTS_SAVEPATH = "generator.h5"
D_WEIGHTS_SAVEPATH = "discriminator.h5"

def train2():
    x_train = load_images("./generate_images/256images.pickle")
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
    embeddings = load_embedings("./generate_images/char-CNN-RNN-embeddings.pickle")
    embeddings_test = load_embedings("./flowers/test/char-CNN-RNN-embeddings.pickle")
    
    discriminator2 = Discriminator2()
    d_opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)
    discriminator2.compile(loss="binary_crossentropy", optimizer=d_opt)
    
    generator = Generator()
    g_opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)
    generator.compile(loss="binary_crossentropy", optimizer=g_opt)
    
    generator.load_weights(G_WEIGHTS_SAVEPATH)
    
    generator2 = Generator2()
    g_opt = Adam(lr=0.0002, beta_1=0.5, beta_2=0.999)
    generator2.compile(loss="binary_crossentropy", optimizer=g_opt)
    
    compressed_embed = Compressed_embed()
    compressed_embed.compile(loss='binary_crossentropy', optimizer="adam")
    
    
    combined_model = adv_model(generator2, discriminator2,generator)
    combined_model.compile(loss=['binary_crossentropy',KL_loss], loss_weights=[1.0, 2.0], optimizer=g_opt,metrics=None)

    real_label = np.ones((BATCH_SIZE, 1)) * 0.9
    fake_label = np.ones((BATCH_SIZE, 1)) * 0.1
    
    for epoch in range(NUM_EPOCH):
        gen_loss=[]
        dis_loss=[]
        
        num_batches = int(x_train.shape[0] / BATCH_SIZE)
        for index in range(num_batches):
            #noise = np.array([np.random.uniform(0,1,100) for _ in range(BATCH_SIZE)])
            noise = np.random.randn(BATCH_SIZE, 100)
            image_batch = x_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            embedding_batch = embeddings[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            
            in_fake_images, _ = generator.predict([embedding_batch, noise], verbose=3)
            out_fake_images, _ = generator2.predict([embedding_batch, in_fake_images], verbose=3)
            
#             if index % 100 == 0:
#                 z_noise2 = np.random.normal(0, 1, size=(BATCH_SIZE, 100))
#                 embedding_batch = embeddings_test[0:32]
#                 fake_images, _ = generator.predict([embedding_batch, z_noise2])
#                 fake_images = fake_images*125.7 + 127.5
#                 # Save image
#                 Image.fromarray(np.uint8(fake_images[0])).save("./generate_images/images4/%depoch_%d.jpg"%(epoch, index))
            
            compressed_embeddings = compressed_embed.predict_on_batch(embedding_batch)
            compressed_embeddings = np.reshape(compressed_embeddings, (-1, 1, 1, 128))
            compressed_embeddings = np.tile(compressed_embeddings, (1, 4, 4, 1))
            

            real_d_loss = discriminator2.train_on_batch([image_batch, compressed_embeddings], np.reshape(real_label, (BATCH_SIZE, 1)))
            fake_d_loss = discriminator2.train_on_batch([out_fake_images, compressed_embeddings],np.reshape(fake_label, (BATCH_SIZE, 1)))
            wrong_d_loss = discriminator2.train_on_batch([image_batch[:(BATCH_SIZE - 1)], compressed_embeddings[1:]],np.reshape(fake_label[1:], (BATCH_SIZE-1, 1)))
            
            d_loss = 0.5 * np.add(real_d_loss, 0.5 * np.add(wrong_d_loss, fake_d_loss))
                                  
            
            #g_loss = combined_model.train_on_batch([embedding_batch, noise, compressed_embeddings],[np.ones((BATCH_SIZE, 1)) * 0.9, np.ones((BATCH_SIZE, 256)) * 0.9])
            g_loss = combined_model.train_on_batch([embedding_batch, noise, compressed_embeddings],[np.ones((BATCH_SIZE, 1)) * 0.9, np.ones((BATCH_SIZE, 256)) * 0.9])
            print("g_loss:{}".format(g_loss))
            
            dis_loss.append(d_loss)
            gen_loss.append(g_loss)

        if epoch % 2 == 0:
            #noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
            noise = np.random.randn(BATCH_SIZE, 100)
            embedding_batch = embeddings_test[0:32]
            
            in_fake_images, _ = generator.predict([embedding_batch, noise], verbose=3)
            out_fake_images, _ = generator2.predict([embedding_batch, in_fake_images], verbose=3)
            
            for i, img in enumerate(out_fake_images[:10]):
                Image.fromarray(np.uint8(img[0])).save("./generate_images/stage2/%depoch_%d.jpg"%(epoch, i))
            
    generator2.save_weights("stage2_gen.h5")
    discriminator2.save_weights("stage2_dis.h5")
                                  

In [10]:
train()

Number of batches: 3517


IndexError: index 2 is out of bounds for axis 0 with size 2

In [15]:
with open("../stackGAN-pre/Data/flowers/test/filenames.pickle", "rb") as f:
    filenames = pickle.load(f)

with open("../stackGAN-pre/Data/flowers/test/class_info.pickle", "rb") as f:
    cl= pickle.load(f)

with open("../stackGAN-pre/Data/flowers/test/char-CNN-RNN-embeddings.pickle","rb") as f:
        embeddings_test = pickle.load(f,encoding="latin")

embeddings = np.array(embeddings_test)
# with open("example-skipthoughts.pickle","rb") as f:
#     embeddings_train = pickle.load(f,encoding="latin")
          
# embeddings = np.array(embeddings_train)
# print(embeddings.shape)
#embeddings.reshape(1,5,-1)
# embeddings_train = np.array(embeddings_train)

with open("../stackGAN-pre/Data/flowers/test/304images.pickle", "rb") as f:
    images = pickle.load(f)
    

g = Generator()
g.load_weights("generator.h5")
num = 60
print(cl[60])
print(filenames[60])
# for i in range(5):
#     z_noise2 = np.random.normal(0, 1, size=(32, 100))
#     fake_images, _ = g.predict([embeddings[i], z_noise2])
#     fake_images = fake_images*125.7 + 127.5
#     Image.fromarray(np.uint8(fake_images[5])).save("./confirm/skipthouts/num%d.jpg"%(i))      

# z_noise2 = np.random.normal(0, 1, size=(32, 100))
# fake_images, _ = g.predict([embeddings[0:32,0], z_noise2])
# print(fake_images.shape)
print(embeddings.shape)
for i in range(32):
    z_noise2 = np.random.normal(0, 1, size=(32, 100))
    fake_images, _ = g.predict([embeddings[0:32,0], z_noise2])
    fake_images = fake_images*125.7 + 127.5
    Image.fromarray(np.uint8(fake_images[i])).save("./generate_images/confirm/t/num%d.jpg"%(i))
# embedding_batch = embeddings[0:1]
# for i in range(10):
# # # Save image
#     for j in range(embeddings.shape[0]):
#         if not os.path.exists(str(i)):
#             os.mkdir(str(i))
#             Image.fromarray(np.uint8(fake_images[i,j])).save("./%d/num%d_%d.jpg"%(i,i,j))
#Image.fromarray(np.uint8(fake_images[1])).save("result3.jpg")

20
jpg/image_04948
(1155, 10, 1024)


In [1]:
import matplotlib.pyplot as plt

with open("flowers/test/filenames.pickle", "rb") as f:
    filenames = pickle.load(f)


with open("flowers/test/char-CNN-RNN-embeddings.pickle","rb") as f:
    emb = pickle.load(f,encoding="latin")
        
    emb = np.array(emb)

compressed_embed = Compressed_embed()
compressed_embed.compile(loss='binary_crossentropy', optimizer="adam")

compressed_embedding = compressed_embed.predict(emb[448])
compressed_embedding = compressed_embedding.reshape((1,10,128))
compressed_embeddings = np.reshape(compressed_embedding, (-1, 1, 1, 128))
compressed_embeddings = np.tile(compressed_embeddings, (1, 4, 4, 1))

real_img = np.array(Image.open("D:/Workspace/DeepLearning/GAN/stackGAN-pre/Data/flowers/jpg/image_06149.jpg").resize((64,64))).reshape(1,64,64,3)

fake_img = np.array(Image.open("D:/Workspace/DeepLearning/GAN/StackGAN/generate_images/confirm/cl19/num1.jpg")).reshape(1,64,64,3)

discriminator = Discriminator()
discriminator.load_weights("discriminator.h5")

# result = discriminator.predict([fake_img, compressed_embeddings])
# print(result)
plt.imshow(fake_img[0])

NameError: name 'pickle' is not defined

# FID

In [21]:
import fid
generator = Generator()
z_noise2 = np.random.normal(0, 1, size=(32, 100))

with open("./Data/traning/64images.pickle", "rb") as f:
    images = pickle.load(f)

fd = fid.FrechetInceptionDistance(generator, (0,1)) 
gan_fid = fd(images, z_noise2)
print(gan_fid)

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'B') according to the casting rule ''same_kind''